## Dependencies

In [1]:
import os
import pickle
import sys
import pandas as pd

# spotify 
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import spotipy.util as util

from dotenv import load_dotenv

Loading dataset

# Connect to spotify

Configure environment variable

In [2]:
# take environment variables from .env.  
load_dotenv()

# changing current directory to read dataset
os.chdir(os.getenv("DATASET_PATH"))

In [3]:
# configuring client_id and client_secret
client_id = os.environ["SPOTIFYREC_CLIENT_ID"]
client_secret= os.environ["SPOTIFYREC_CLIENT_SECRET"]

Setting max columns and rows

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

Authenticating

In [5]:
auth = SpotifyClientCredentials(
    client_id=client_id, 
    client_secret=client_secret
)

Creating token

In [6]:
token = util.prompt_for_user_token(
    'playlist-modify-private', 
     client_id= client_id, 
     client_secret=client_secret, 
     redirect_uri=os.environ["SPOTIFY_URI"]                              
)

In [7]:
sp = spotipy.Spotify(auth=token)

In [8]:
# getting user created playlists
user_created_playlists = {
    i['name']:i['id'] for i in sp.current_user_playlists()['items']
}

In [9]:
user_created_playlists

{'Beatles enhanced': '6GeZV5v80WsMrjMc3C9bEo',
 'Beatles': '6cdgZqKsbzS4D6u1CX8xQi',
 'Mix of artists': '0shdRzW2DhDUGViKoOF6px',
 'Your Top Songs 2022': '37i9dQZF1F0sijgNaJdgit',
 'Arabic': '3KmKRBAbrT1uNs0oamMu1L',
 'Vibes': '5TVhAkpfczVXev3mMTWIQM',
 'ECN503 - Podcasts for Essay': '7u21UI3Xe97wn6qIiJpl0f'}

Now we need to create a dataframe with the specified playlist

In [10]:
# inputs
playlists = ['Beatles','Mix of artists']
playlist_id = user_created_playlists

# creating lists of date song added, track id, tracks and artists
list_of_playlists = []
for playlist_name in playlists:
    date_added = [i['added_at'] for i in sp.playlist(playlist_id[playlist_name])['tracks']['items']]
    track_ids = [i['track']['id'] for i in sp.playlist(playlist_id[playlist_name])['tracks']['items']]
    tracks = [i['track']['name'] for i in sp.playlist(playlist_id[playlist_name])['tracks']['items']]
    artist_names = [i['track']['artists'][0]['name'] for i in sp.playlist(playlist_id[playlist_name])['tracks']['items']]
    
    # appending each playlist
    list_of_playlists.append(
        {
       'date_added':date_added,
       'track_ids':track_ids,
       'tracks':tracks,
       'artist_names':artist_names 
       } 
    )


In [11]:
list_of_playlists

[{'date_added': ['2022-11-22T00:18:54Z',
   '2022-11-22T00:18:57Z',
   '2022-11-22T00:20:28Z'],
  'track_ids': ['6dGnYIeXmHdcikdzNNDMm2',
   '7iN1s7xHE4ifF5povM6A48',
   '1eT2CjXwFXNx6oY5ydvzKU'],
  'tracks': ['Here Comes The Sun - Remastered 2009',
   'Let It Be - Remastered 2009',
   'Hey Jude'],
  'artist_names': ['The Beatles', 'The Beatles', 'The Beatles']},
 {'date_added': ['2022-12-03T18:50:18Z',
   '2022-12-03T19:35:20Z',
   '2022-12-03T19:36:00Z'],
  'track_ids': ['7J1uxwnxfQLu4APicE5Rnj',
   '4JehYebiI9JE8sR8MisGVb',
   '0wwPcA6wtMf6HUMpIRdeP7'],
  'tracks': ['Billie Jean', 'Halo', 'Hotline Bling'],
  'artist_names': ['Michael Jackson', 'Beyoncé', 'Drake']}]

In [12]:
# putting together dataframe
list_of_playists_df = []
for playlist in list_of_playlists:
    playlist_df = pd.DataFrame(
                {
                'id': playlist['track_ids'],
                'tracks': playlist['tracks'],
                'artist_names': playlist['artist_names'],
                'date_added': playlist['date_added']
                }
            )
            
    
    playlist_df['date_added'] = pd.to_datetime(playlist_df['date_added'])  
    list_of_playists_df.append(playlist_df)

Saving playlist

In [13]:
with open('list_of_playists_df.pkl', 'wb') as file:
    pickle.dump(list_of_playists_df, file)

In [14]:
# displaying dataframe
for df in list_of_playists_df:
    display(df)

,id,tracks,artist_names,date_added
0,6dGnYIeXmHdcikdzNNDMm2,Here Comes The Sun - Remastered 2009,The Beatles,2022-11-22 00:18:54+00:00
1,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,2022-11-22 00:18:57+00:00
2,1eT2CjXwFXNx6oY5ydvzKU,Hey Jude,The Beatles,2022-11-22 00:20:28+00:00


,id,tracks,artist_names,date_added
0,7J1uxwnxfQLu4APicE5Rnj,Billie Jean,Michael Jackson,2022-12-03 18:50:18+00:00
1,4JehYebiI9JE8sR8MisGVb,Halo,Beyoncé,2022-12-03 19:35:20+00:00
2,0wwPcA6wtMf6HUMpIRdeP7,Hotline Bling,Drake,2022-12-03 19:36:00+00:00
